In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 61 kB 498 kB/s 
     |████████████████████████████████| 596 kB 37.7 MB/s 
     |████████████████████████████████| 895 kB 32.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [3]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
import numpy as np

model_name = "roberta-large-mnli" #"ishan/bert-base-uncased-mnli"

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [39]:
# val_df = pd.read_csv("/content/drive/MyDrive/NLP/paraphrased_dataset.csv")
val_df = pd.read_csv("/content/drive/MyDrive/NLP/paraphrased_mismatched_dataset_v2.csv")

In [40]:
val_df

,Unnamed: 0,Unnamed: 0.1,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1dash,sentence2dash
0,0,0,"['contradiction', 'contradiction', 'contradict...",letters,contradiction,75290c,75290,Your contribution helped make it possible for ...,( ( Your contribution ) ( ( helped ( make ( it...,(ROOT (S (NP (PRP$ Your) (NN contribution)) (V...,Your contributions were of no help with our st...,( ( Your contributions ) ( ( were ( of ( ( no ...,(ROOT (S (NP (PRP$ Your) (NNS contributions)) ...,NaN,Your contributions were of no help with our st...
1,1,1,"['contradiction', 'contradiction', 'contradict...",verbatim,contradiction,133794c,133794,"The answer has nothing to do with their cause,...",( ( ( ( ( ( The answer ) ( ( ( ( has nothing )...,(ROOT (S (S (NP (DT The) (NN answer)) (VP (VBZ...,Dictionaries are indeed exercises in bi-unique...,( Dictionaries ( ( ( are indeed ) ( exercises ...,(ROOT (S (NP (NNS Dictionaries)) (VP (VBP are)...,NaN,"In fact, dictionaries are exercises in bi-uniq..."
2,2,2,"['contradiction', 'neutral', 'entailment', 'en...",verbatim,entailment,3628c,3628,We serve a classic Tuscan meal that includes ...,( We ( ( serve ( ( a ( classic ( Tuscan meal )...,(ROOT (S (NP (PRP We)) (VP (VBP serve) (NP (NP...,We serve a meal of Florentine terrine.,( We ( ( serve ( ( a meal ) ( of ( Florentine ...,(ROOT (S (NP (PRP We)) (VP (VBP serve) (NP (NP...,"We serve a traditional Tuscan dinner, which in...",We serve a dinner of Florentine terrine.
3,3,3,"['contradiction', 'contradiction', 'contradict...",letters,contradiction,89411c,89411,"A few months ago, Carl Newton and I wrote a le...","( ( ( A ( few months ) ) ago ) ( , ( ( ( ( Car...",(ROOT (S (ADVP (NP (DT A) (JJ few) (NNS months...,Carl Newton and I have never had any other pre...,( ( ( ( Carl Newton ) and ) I ) ( ( ( have nev...,(ROOT (S (NP (NP (NNP Carl) (NNP Newton)) (CC ...,NaN,NaN
4,4,4,"['entailment', 'entailment', 'entailment', 'en...",facetoface,entailment,136158e,136158,"I was on this earth you know, I've lived on th...",( I ( ( was ( on ( ( this earth ) ( you ( know...,(ROOT (S (NP (PRP I)) (VP (VBD was) (PP (IN on...,I don't yet know the reason why I have lived o...,( I ( ( ( ( do n't ) yet ) ( ( know ( the reas...,(ROOT (S (NP (PRP I)) (VP (VBP do) (RB n't) (A...,NaN,I do not yet know the reason I have lived on e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,"['contradiction', 'contradiction', 'neutral', ...",facetoface,contradiction,47686c,47686,Do you watch that?,( ( ( Do you ) ( watch that ) ) ? ),(ROOT (SQ (VBP Do) (NP (PRP you)) (VP (VB watc...,Can you see?,( ( ( Can you ) see ) ? ),(ROOT (SQ (MD Can) (NP (PRP you)) (VP (VB see)...,Do you watch this?,NaN
9996,9996,9996,"['contradiction', 'contradiction', 'contradict...",verbatim,contradiction,126829c,126829,"To a Western ear, the most predictable of lang...","( ( ( ( ( To ( ( ( a ( Western ear ) ) , ) ( (...",(ROOT (SINV (S (S (PP (TO To) (NP (NP (DT a) (...,"To the Western ear, the least predictable of l...","( ( To ( the ( Western ear ) ) ) ( , ( ( ( the...",(ROOT (S (PP (TO To) (NP (DT the) (JJ Western)...,NaN,NaN
9997,9997,9997,"['contradiction', 'contradiction', 'contradict...",nineeleven,contradiction,37368c,37368,The recorder captured the sounds of loud thump...,( ( The recorder ) ( ( ( ( captured ( ( the so...,(ROOT (S (NP (DT The) (NN recorder)) (VP (VP (...,The recorder didn't capture any of the sounds.,( ( The recorder ) ( ( ( did n't ) ( capture (...,(ROOT (S (NP (DT The) (NN recorder)) (VP (VBD ...,The recorder captured the sounds of loud thump...,The recorder did not capture any of the sounds.
9998,9998,9998,"['neutral', 'neutral', 'entailment', 'neutral'...",facetoface,neutral,97460n,97460,That's a good attitude!,( That ( ( 's ( a ( good attitude ) ) ) ! ) ),(ROOT (S (NP (DT That)) (VP (VBZ 's) (NP (DT a...,"You feel good about this, don't you?","( ( Yo

In [41]:
val_df.dtypes

Unnamed: 0                 int64
Unnamed: 0.1               int64
annotator_labels          object
genre                     object
gold_label                object
pairID                    object
promptID                   int64
sentence1                 object
sentence1_binary_parse    object
sentence1_parse           object
sentence2                 object
sentence2_binary_parse    object
sentence2_parse           object
sentence1dash             object
sentence2dash             object
dtype: object

In [42]:
# val_df['sentence1'].isnull().values.any()

In [43]:
val_df['sentence1'] = val_df['sentence1'].astype(str)
val_df['sentence2'] = val_df['sentence2'].astype(str)

In [44]:
val_df['sentence1dash'] = val_df['sentence1dash'].astype(str)
val_df['sentence2dash'] = val_df['sentence2dash'].astype(str)

In [45]:
val_df

,Unnamed: 0,Unnamed: 0.1,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1dash,sentence2dash
0,0,0,"['contradiction', 'contradiction', 'contradict...",letters,contradiction,75290c,75290,Your contribution helped make it possible for ...,( ( Your contribution ) ( ( helped ( make ( it...,(ROOT (S (NP (PRP$ Your) (NN contribution)) (V...,Your contributions were of no help with our st...,( ( Your contributions ) ( ( were ( of ( ( no ...,(ROOT (S (NP (PRP$ Your) (NNS contributions)) ...,nan,Your contributions were of no help with our st...
1,1,1,"['contradiction', 'contradiction', 'contradict...",verbatim,contradiction,133794c,133794,"The answer has nothing to do with their cause,...",( ( ( ( ( ( The answer ) ( ( ( ( has nothing )...,(ROOT (S (S (NP (DT The) (NN answer)) (VP (VBZ...,Dictionaries are indeed exercises in bi-unique...,( Dictionaries ( ( ( are indeed ) ( exercises ...,(ROOT (S (NP (NNS Dictionaries)) (VP (VBP are)...,nan,"In fact, dictionaries are exercises in bi-uniq..."
2,2,2,"['contradiction', 'neutral', 'entailment', 'en...",verbatim,entailment,3628c,3628,We serve a classic Tuscan meal that includes ...,( We ( ( serve ( ( a ( classic ( Tuscan meal )...,(ROOT (S (NP (PRP We)) (VP (VBP serve) (NP (NP...,We serve a meal of Florentine terrine.,( We ( ( serve ( ( a meal ) ( of ( Florentine ...,(ROOT (S (NP (PRP We)) (VP (VBP serve) (NP (NP...,"We serve a traditional Tuscan dinner, which in...",We serve a dinner of Florentine terrine.
3,3,3,"['contradiction', 'contradiction', 'contradict...",letters,contradiction,89411c,89411,"A few months ago, Carl Newton and I wrote a le...","( ( ( A ( few months ) ) ago ) ( , ( ( ( ( Car...",(ROOT (S (ADVP (NP (DT A) (JJ few) (NNS months...,Carl Newton and I have never had any other pre...,( ( ( ( Carl Newton ) and ) I ) ( ( ( have nev...,(ROOT (S (NP (NP (NNP Carl) (NNP Newton)) (CC ...,nan,nan
4,4,4,"['entailment', 'entailment', 'entailment', 'en...",facetoface,entailment,136158e,136158,"I was on this earth you know, I've lived on th...",( I ( ( was ( on ( ( this earth ) ( you ( know...,(ROOT (S (NP (PRP I)) (VP (VBD was) (PP (IN on...,I don't yet know the reason why I have lived o...,( I ( ( ( ( do n't ) yet ) ( ( know ( the reas...,(ROOT (S (NP (PRP I)) (VP (VBP do) (RB n't) (A...,nan,I do not yet know the reason I have lived on e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,"['contradiction', 'contradiction', 'neutral', ...",facetoface,contradiction,47686c,47686,Do you watch that?,( ( ( Do you ) ( watch that ) ) ? ),(ROOT (SQ (VBP Do) (NP (PRP you)) (VP (VB watc...,Can you see?,( ( ( Can you ) see ) ? ),(ROOT (SQ (MD Can) (NP (PRP you)) (VP (VB see)...,Do you watch this?,nan
9996,9996,9996,"['contradiction', 'contradiction', 'contradict...",verbatim,contradiction,126829c,126829,"To a Western ear, the most predictable of lang...","( ( ( ( ( To ( ( ( a ( Western ear ) ) , ) ( (...",(ROOT (SINV (S (S (PP (TO To) (NP (NP (DT a) (...,"To the Western ear, the least predictable of l...","( ( To ( the ( Western ear ) ) ) ( , ( ( ( the...",(ROOT (S (PP (TO To) (NP (DT the) (JJ Western)...,nan,nan
9997,9997,9997,"['contradiction', 'contradiction', 'contradict...",nineeleven,contradiction,37368c,37368,The recorder captured the sounds of loud thump...,( ( The recorder ) ( ( ( ( captured ( ( the so...,(ROOT (S (NP (DT The) (NN recorder)) (VP (VP (...,The recorder didn't capture any of the sounds.,( ( The recorder ) ( ( ( did n't ) ( capture (...,(ROOT (S (NP (DT The) (NN recorder)) (VP (VBD ...,The recorder captured the sounds of loud thump...,The recorder did not capture any of the sounds.
9998,9998,9998,"['neutral', 'neutral', 'entailment', 'neutral'...",facetoface,neutral,97460n,97460,That's a good attitude!,( That ( ( 's ( a ( good attitude ) ) ) ! ) ),(ROOT (S (NP (DT That)) (VP (VBZ 's) (NP (DT a...,"You feel good about this, don't you?","( ( Yo

In [46]:
val_df = val_df[val_df['gold_label'] != '-']
val_df

,Unnamed: 0,Unnamed: 0.1,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1dash,sentence2dash
0,0,0,"['contradiction', 'contradiction', 'contradict...",letters,contradiction,75290c,75290,Your contribution helped make it possible for ...,( ( Your contribution ) ( ( helped ( make ( it...,(ROOT (S (NP (PRP$ Your) (NN contribution)) (V...,Your contributions were of no help with our st...,( ( Your contributions ) ( ( were ( of ( ( no ...,(ROOT (S (NP (PRP$ Your) (NNS contributions)) ...,nan,Your contributions were of no help with our st...
1,1,1,"['contradiction', 'contradiction', 'contradict...",verbatim,contradiction,133794c,133794,"The answer has nothing to do with their cause,...",( ( ( ( ( ( The answer ) ( ( ( ( has nothing )...,(ROOT (S (S (NP (DT The) (NN answer)) (VP (VBZ...,Dictionaries are indeed exercises in bi-unique...,( Dictionaries ( ( ( are indeed ) ( exercises ...,(ROOT (S (NP (NNS Dictionaries)) (VP (VBP are)...,nan,"In fact, dictionaries are exercises in bi-uniq..."
2,2,2,"['contradiction', 'neutral', 'entailment', 'en...",verbatim,entailment,3628c,3628,We serve a classic Tuscan meal that includes ...,( We ( ( serve ( ( a ( classic ( Tuscan meal )...,(ROOT (S (NP (PRP We)) (VP (VBP serve) (NP (NP...,We serve a meal of Florentine terrine.,( We ( ( serve ( ( a meal ) ( of ( Florentine ...,(ROOT (S (NP (PRP We)) (VP (VBP serve) (NP (NP...,"We serve a traditional Tuscan dinner, which in...",We serve a dinner of Florentine terrine.
3,3,3,"['contradiction', 'contradiction', 'contradict...",letters,contradiction,89411c,89411,"A few months ago, Carl Newton and I wrote a le...","( ( ( A ( few months ) ) ago ) ( , ( ( ( ( Car...",(ROOT (S (ADVP (NP (DT A) (JJ few) (NNS months...,Carl Newton and I have never had any other pre...,( ( ( ( Carl Newton ) and ) I ) ( ( ( have nev...,(ROOT (S (NP (NP (NNP Carl) (NNP Newton)) (CC ...,nan,nan
4,4,4,"['entailment', 'entailment', 'entailment', 'en...",facetoface,entailment,136158e,136158,"I was on this earth you know, I've lived on th...",( I ( ( was ( on ( ( this earth ) ( you ( know...,(ROOT (S (NP (PRP I)) (VP (VBD was) (PP (IN on...,I don't yet know the reason why I have lived o...,( I ( ( ( ( do n't ) yet ) ( ( know ( the reas...,(ROOT (S (NP (PRP I)) (VP (VBP do) (RB n't) (A...,nan,I do not yet know the reason I have lived on e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,"['contradiction', 'contradiction', 'neutral', ...",facetoface,contradiction,47686c,47686,Do you watch that?,( ( ( Do you ) ( watch that ) ) ? ),(ROOT (SQ (VBP Do) (NP (PRP you)) (VP (VB watc...,Can you see?,( ( ( Can you ) see ) ? ),(ROOT (SQ (MD Can) (NP (PRP you)) (VP (VB see)...,Do you watch this?,nan
9996,9996,9996,"['contradiction', 'contradiction', 'contradict...",verbatim,contradiction,126829c,126829,"To a Western ear, the most predictable of lang...","( ( ( ( ( To ( ( ( a ( Western ear ) ) , ) ( (...",(ROOT (SINV (S (S (PP (TO To) (NP (NP (DT a) (...,"To the Western ear, the least predictable of l...","( ( To ( the ( Western ear ) ) ) ( , ( ( ( the...",(ROOT (S (PP (TO To) (NP (DT the) (JJ Western)...,nan,nan
9997,9997,9997,"['contradiction', 'contradiction', 'contradict...",nineeleven,contradiction,37368c,37368,The recorder captured the sounds of loud thump...,( ( The recorder ) ( ( ( ( captured ( ( the so...,(ROOT (S (NP (DT The) (NN recorder)) (VP (VP (...,The recorder didn't capture any of the sounds.,( ( The recorder ) ( ( ( did n't ) ( capture (...,(ROOT (S (NP (DT The) (NN recorder)) (VP (VBD ...,The recorder captured the sounds of loud thump...,The recorder did not capture any of the sounds.
9998,9998,9998,"['neutral', 'neutral', 'entailment', 'neutral'...",facetoface,neutral,97460n,97460,That's a good attitude!,( That ( ( 's ( a ( good attitude ) ) ) ! ) ),(ROOT (S (NP (DT That)) (VP (VBZ 's) (NP (DT a...,"You feel good about this, don't you?","( ( Yo

In [49]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

class MNLIDataRoberta(Dataset):

  def __init__(self, val_df):
    self.label_dict = {'entailment': 2, 'contradiction': 0, 'neutral': 1}

    # self.train_df = train_df
    self.val_df = val_df

    self.base_path = '/content/'
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    # self.train_data = None
    self.val_data = None
    self.paraphrased_data = None
    self.init_data()

  def get_tokenizer(self):
    return self.tokenizer

  def init_data(self):
    # Saving takes too much RAM
    #
    # if os.path.exists(os.path.join(self.base_path, 'train_data.pkl')):
    #   print("Found training data")
    #   with open(os.path.join(self.base_path, 'train_data.pkl'), 'rb') as f:
    #     self.train_data = pickle.load(f)
    # else:
    #   self.train_data = self.load_data(self.train_df)
    #   with open(os.path.join(self.base_path, 'train_data.pkl'), 'wb') as f:
    #     pickle.dump(self.train_data, f)
    # if os.path.exists(os.path.join(self.base_path, 'val_data.pkl')):
    #   print("Found val data")
    #   with open(os.path.join(self.base_path, 'val_data.pkl'), 'rb') as f:
    #     self.val_data = pickle.load(f)
    # else:
    #   self.val_data = self.load_data(self.val_df)
    #   with open(os.path.join(self.base_path, 'val_data.pkl'), 'wb') as f:
    #     pickle.dump(self.val_data, f)
    # self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)
    self.paraphrased_data = self.load_paraphrased_data(self.val_df)

  def load_paraphrased_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    paraphrased_premise_list = df['sentence1dash'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    paraphrased_hypothesis_list = df['sentence2dash'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, para_premise, hypothesis, para_hypothesis, label) in zip(premise_list, paraphrased_premise_list, hypothesis_list, paraphrased_hypothesis_list, label_list):

      # print(premise, para_premise, hypothesis, para_hypothesis)
      if para_hypothesis != 'nan':
        encoded_values = self.tokenizer.encode_plus(premise, para_hypothesis, return_token_type_ids=True, return_attention_mask=True)
        token_ids.append(torch.tensor(encoded_values['input_ids']))
        seg_ids.append(torch.tensor(encoded_values['token_type_ids']))
        mask_ids.append(torch.tensor(encoded_values['attention_mask']))
        y.append(self.label_dict[label])

      if para_premise != 'nan':
        encoded_values = self.tokenizer.encode_plus(para_premise, hypothesis, return_token_type_ids=True, return_attention_mask=True)
        token_ids.append(torch.tensor(encoded_values['input_ids']))
        seg_ids.append(torch.tensor(encoded_values['token_type_ids']))
        mask_ids.append(torch.tensor(encoded_values['attention_mask']))
        y.append(self.label_dict[label])

      if para_hypothesis != 'nan' and para_premise != 'nan':
        encoded_values = self.tokenizer.encode_plus(para_premise, para_hypothesis, return_token_type_ids=True, return_attention_mask=True)
        token_ids.append(torch.tensor(encoded_values['input_ids']))
        seg_ids.append(torch.tensor(encoded_values['token_type_ids']))
        mask_ids.append(torch.tensor(encoded_values['attention_mask']))
        y.append(self.label_dict[label])

    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['sentence1'].to_list()
    hypothesis_list = df['sentence2'].to_list()
    label_list = df['gold_label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      # premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      # hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      # pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      # premise_len = len(premise_id)
      # hypothesis_len = len(hypothesis_id)

      # segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      # attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values
      # print(premise, hypothesis)
      if premise != 'nan' and hypothesis != 'nan':
        encoded_values = self.tokenizer.encode_plus(premise, hypothesis, return_token_type_ids=True, return_attention_mask=True)
        token_ids.append(torch.tensor(encoded_values['input_ids']))
        seg_ids.append(torch.tensor(encoded_values['token_type_ids']))
        mask_ids.append(torch.tensor(encoded_values['attention_mask']))
        y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    # train_loader = DataLoader(
    #   self.train_data,
    #   shuffle=shuffle,
    #   batch_size=batch_size
    # )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    paraphrased_loader = DataLoader(
        self.paraphrased_data,
        shuffle=shuffle,
        batch_size=batch_size
    )

    return val_loader, paraphrased_loader

In [50]:
mnli_dataset = MNLIDataRoberta(val_df)

9832
8165


In [51]:
val_loader, paraphrased_loader = mnli_dataset.get_data_loaders(batch_size=32)

In [52]:
print(len(paraphrased_loader), len(val_loader))

256 308


In [53]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [54]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 355,362,819 trainable parameters


In [55]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [56]:
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  # print(pred_flat)
  # print(labels_flat)
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [57]:
model.eval()
total_val_acc  = 0
total_val_loss = 0
total_val_acc_para = 0
total_val_loss_para = 0
# output_orig = {}
# output_predict = {}
# ind = 0
with torch.no_grad():
  for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
    # optimizer.zero_grad()
    pair_token_ids = pair_token_ids.to(device)
    mask_ids = mask_ids.to(device)
    seg_ids = seg_ids.to(device)
    labels = y.to(device)

    # prediction = model(pair_token_ids, mask_ids, seg_ids)
    result = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels,
                             return_dict=True)
  
    loss = result.loss
    logits = result.logits
    # print(logits.shape)
    logits = logits.detach().cpu().numpy()
    label_ids = labels.to('cpu').numpy()
        
        # loss = criterion(prediction, labels)
    acc = flat_accuracy(logits, label_ids)
    # print(acc, loss)

    total_val_loss += loss.item()
    total_val_acc  += acc
  
  for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(paraphrased_loader):
    # optimizer.zero_grad()
    pair_token_ids = pair_token_ids.to(device)
    mask_ids = mask_ids.to(device)
    seg_ids = seg_ids.to(device)
    labels = y.to(device)

    result = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels,
                             return_dict=True)
  
    loss = result.loss
    logits = result.logits
    # print(logits.shape)
    logits = logits.detach().cpu().numpy()
    label_ids = labels.to('cpu').numpy()
    # pred_flat = np.argmax(logits, axis=1).flatten()
        
        # loss = criterion(prediction, labels)
    acc = flat_accuracy(logits, label_ids)

    total_val_loss_para += loss.item()
    total_val_acc_para  += acc
    # output_predict[ind] = {'input_ids':pair_token_ids, 'label_real': label_ids[0], 'label_predict':pred_flat[0]}
    # ind += 1

val_acc  = total_val_acc/len(val_loader)
val_loss = total_val_loss/len(val_loader)
val_acc_para = total_val_acc_para/len(paraphrased_loader)
val_loss_para = total_val_loss_para/len(paraphrased_loader)

In [58]:
print(f'val_loss: {val_loss:.4f} val_acc: {val_acc:.4f} | val_loss_para: {val_loss_para:.4f} val_acc: {val_acc_para:.4f}')

val_loss: 0.2878 val_acc: 0.9015 | val_loss_para: 0.3129 val_acc: 0.8935


In [ ]:
print(f'val_loss: {val_loss:.4f} val_acc: {val_acc:.4f} | val_loss_para: {val_loss_para:.4f} val_acc: {val_acc_para:.4f}')

val_loss: 0.2758 val_acc: 0.9035 | val_loss_para: 0.2968 val_acc: 0.8996


In [ ]:
print(f'val_loss: {val_loss:.4f} val_acc: {val_acc:.4f} | val_loss_para: {val_loss_para:.4f} val_acc: {val_acc_para:.4f}')

val_loss: 0.2893 val_acc: 0.9012 | val_loss_para: 0.6543 val_acc: 0.7630


In [ ]:
eval_df = pd.DataFrame([],columns=['sentence', 'label', 'pred'])
tokenizer = mnli_dataset.get_tokenizer()
for item in list(output_predict.values()):
    sent = tokenizer.decode(item['input_ids'][0],skip_special_tokens=False)
    print(sent)
    pred = item['label_predict']
    label = item['label_real']
    new_row = {'sentence':sent,'label':label, 'pred': pred}
    eval_df = eval_df.append(new_row, ignore_index=True)

In [ ]:
eval_df

In [ ]:
eval_df.to_csv("/content/drive/MyDrive/NLP/Project/eval_df.csv")

In [ ]:
wrong_preds = eval_df[eval_df['label'] != eval_df['pred']]
wrong_preds

In [ ]:
wrong_preds.to_csv("/content/drive/MyDrive/NLP/Project/wrong_preds.csv")

In [ ]:
print(f'val_loss: {val_loss:.4f} val_acc: {val_acc:.4f} | val_loss_para: {val_loss_para:.4f} val_acc_para: {val_acc_para:.4f}')